In [2]:
import os
import torch

In [3]:
import re #thư viện dùng xử lý chuỗi
import unicodedata #thư viện dùng decode câu thành unicode8
import spacy #thư viện giúp xử lý, trích xuất thông tin từ văn bản 
from tqdm import tqdm #thư viện dùng để tạo thanh triến trình (cho train model)
import pandas as pd #thư viện dùng để tạo ma trận dữ liệu
import numpy as np #thư viện dùng để xử lý mảng nhiều chiều
import random #thư viện tạo số ngẫu nhiên
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator #vẽ biểu đồ wordcloud
import matplotlib.pyplot as plt #vẽ biểu đồ phân tích data
import seaborn as sns #vẽ biểu đồ phân tích data

In [19]:
data = pd.read_csv(r"C:\Users\Admin\Downloads\Grammar Correction\C4_200M.tsv-00000-of-00010", sep='\t', header=None, names=['input', 'output'])

In [20]:
data = data[:2000000]

In [21]:
print(len(data)) #số lượng dòng dữ liệu trong file

2000000


In [22]:
data.describe() #tính số dòng dữ liệu (count), số dòng độc nhất (unique), số từ xuất hiện nhiều nhất (top), số lần xuất hiện của từ nhiều nhất (freq)


,input,output
count,2000000,1999997
unique,1999981,1999997
top,33387.,"Bitcoin goes for $7,094 this morning, accordin..."
freq,5,1


In [23]:
data = data.drop_duplicates() #loại bỏ các dòng trùng lặp

In [24]:
data.describe() #các dòng giống hệt nhau đã được loại bỏ

,input,output
count,2000000,1999997
unique,1999981,1999997
top,33387.,"Bitcoin goes for $7,094 this morning, accordin..."
freq,5,1


In [25]:
data = data.dropna() #loại bỏ các dòng có giá trị null

In [26]:
data.describe() #dòng nào có giá trị null đã bị loại bỏ

,input,output
count,1999997,1999997
unique,1999978,1999997
top,33387.,"Bitcoin goes for $7,094 this morning, accordin..."
freq,5,1


In [27]:
data.rename(columns = {"input": "error", "output": "correct"}, inplace=True) #đổi tên cột

In [28]:
len_1 = [ len(data['correct'].iloc[i]) for i in range(len(data['correct'])) ] #list lưu độ dài của từng câu trong cột correct
len_2= [ len(data['error'].iloc[i]) for i in range(len(data['error'])) ] #list lưu độ dài của từng câu trong cột error

In [29]:
for i, row in enumerate(data['error']): #check từng câu trong cột error
  if row.split(" ") == data['correct'].iloc[i].split(" ") or len(row.split(" ")) >= 30: #nếu câu trong cột error giống câu trong cột correct hoặc độ dài câu error lớn hơn 30 hoặc nhỏ hơn 5
    data.iloc[i] = np.nan #thì gán giá trị null cho dòng đó

In [30]:
data = data.dropna() #loại bỏ các dòng có giá trị null

In [31]:
pd.set_option('display.max_colwidth', None)
print(data.head())#đặt hiển thị tối đa cho cột là không giới hạn

                                                                                            error  \
0                                        Bitcoin is for $7,094 this morning, which CoinDesk says.   
1         The effect of widespread dud targets two face up attack position monsters on the field.   
3                                               Much many brands and sellers still in the market.   
4                                          this is is the latest Maintenance release of Samba 3.6   
5  Fairy Or Not, I'm the Godmother: no just look, but my outfit for taking the part as godmother.   

                                                                                                correct  
0                                          Bitcoin goes for $7,094 this morning, according to CoinDesk.  
1          1. The effect of "widespread dud" targets two face up attack position monsters on the field.  
3                                                          Many brands and 

In [32]:
data.describe() #các dòng giống nhau ở cả 2 cột đã bị loại bỏ

,error,correct
count,1519886,1519886
unique,1519867,1519886
top,33387.,"Bitcoin goes for $7,094 this morning, according to CoinDesk."
freq,5,1


In [33]:
data

,error,correct
0,"Bitcoin is for $7,094 this morning, which CoinDesk says.","Bitcoin goes for $7,094 this morning, according to CoinDesk."
1,The effect of widespread dud targets two face up attack position monsters on the field.,"1. The effect of ""widespread dud"" targets two face up attack position monsters on the field."
3,Much many brands and sellers still in the market.,Many brands and sellers still in the market.
4,this is is the latest Maintenance release of Samba 3.6,This is is the latest maintenance release of Samba 3.6.
5,"Fairy Or Not, I'm the Godmother: no just look, but my outfit for taking the part as godmother.","Fairy Or Not, I'm the Godmother: Not just a look, but my outfit for taking on the role as godmother."
...,...,...
1999995,We can trust this as body is a temporal model for communicating Love.,We can trust that the body is a temporary tool for communicating Love.
1999996,Need : Children undergo a cancer treatment can feel discouraged sad and lost during cancer treatment.,"Need : Children undergoing cancer treatment can feel discouraged, sad, and lost during their cancer treatment."
1999997,The Municipal Waste Association is incorporated not-for-profit organization based on municipal waste management professionals in order to facilitate the sharing of municipal waste reduction and recycling information and expertise.,The Municipal Waste Association is an incorporated not-for-profit organization formed in 1987 by Ontario municipal waste management professionals to facilitate the sharing of municipal waste reduction and recycling information and expertise.
1999998,I need a day off suddenly!,"I need a day off, suddenly!"


In [60]:
data.isna().any() #check xem có dòng nào có giá trị null không

error      False
correct    False
dtype: bool

In [34]:
def decontracted(phrase): #hàm chuyển từ viết tắt thành từ đầy đủ
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"gon na", " going to", phrase)
    phrase = re.sub(r"wan na", " want to", phrase)
    phrase = re.sub(r"gonna", " going to", phrase)
    phrase = re.sub(r"wanna", " want to", phrase)


    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub("\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

def clean_text(t): #hàm bỏ dấu và ký tự unicode

  t = unicodedata.normalize('NFKD', t).encode('ascii', 'ignore').decode('ascii') 
  t = decontracted(t)

  t = re.sub(r'x D', '', t)
  t = re.sub(r': D', '', t)
  t = re.sub(r': P', '', t)

  t = re.sub(r'xD', '', t)
  t = re.sub(r':D', '', t)
  t = re.sub(r':P', '', t)

  if '(' in t and ')' in t: #loại bỏ nội dung trong dấu ngoặc đơn => Hello (world) => Hello
    try:
      t = re.sub(t.split("(")[-1].split(")")[0], '', t)
    except:
      pass
    #t = re.sub("(", '', t)
    #t = re.sub(")", '', t)

  t = re.sub(r'[^A-Za-z;!?.,\-\' ]+', ' ', t) #loại bỏ các emoji, số, ký tự đặc biệt

#chuẩn hóa dấu câu, thêm khoảng trống trước dâu câu. 
  t = re.sub(r'\.+',r' .',t)
  t = re.sub(r'\;+',r' , ',t)
  t = re.sub(r'!+',r' !',t )
  t = re.sub(r'\?+',r' ?',t )
  t = re.sub(r'\-+',r' - ',t )
  t = re.sub(r'\,+',r' , ',t )
  t = re.sub(r'\'+',r" ' ",t)
  t = re.sub(' +', ' ', t)
  t = t.lower()

  return t #trả về văn bản được làm sạch

In [35]:
print(clean_text("I love you!"))
print(clean_text("I love you !"))

i love you !
i love you !


In [36]:
data['correct'] = data['correct'].apply(clean_text) #áp dụng hàm clean_text cho cột correct
data['error'] = data['error'].apply(clean_text) #áp dụng hàm clean_text cho cột error

C:\Users\Admin\AppData\Local\Temp\ipykernel_23152\2720349451.py:37: FutureWarning: Possible nested set at position 1
  t = re.sub(t.split("(")[-1].split(")")[0], '', t)
C:\Users\Admin\AppData\Local\Temp\ipykernel_23152\2720349451.py:37: FutureWarning: Possible nested set at position 22
  t = re.sub(t.split("(")[-1].split(")")[0], '', t)


In [37]:
data[:50] # in ra 50 dòng đầu tiên trong dataframe

,error,correct
0,"bitcoin is for , this morning , which coindesk says .","bitcoin goes for , this morning , according to coindesk ."
1,the effect of widespread dud targets two face up attack position monsters on the field .,. the effect of widespread dud targets two face up attack position monsters on the field .
3,much many brands and sellers still in the market .,many brands and sellers still in the market .
4,this is is the latest maintenance release of samba .,this is is the latest maintenance release of samba . .
5,"fairy or not , i am the godmother no just look , but my outfit for taking the part as godmother .","fairy or not , i am the godmother not just a look , but my outfit for taking on the role as godmother ."
6,watcch as this dodge challenger hellcat gets smoked by a tesla model s - with the drag strip .,watch as this dodge challenger hellcat gets smoked by a tesla model s at the drag strip .
7,"momover , these devices have been proven to help consumers during another company his information .","moreover , these devices are proven to help consumers while another company that information ."
9,worthless involved is supporting for the movement .,get involved and help the movement !
10,mark mohler said in a post on instagram that he and fellow diver kimberley jeffries have confirmed the identity of wonka bee specilis deep blue on wednesday .,"on wednesday , diver mark mohler said in a post on instagram he and fellow diver kimberley jeffries confirmed the identity of the shark as deep blue ."
11,"once done , you could pay for your pumpkins and then leave or take advantage of the great stuff in there .","once done , you could pay for your pumpkins and then leave or take advantage of the great stuff there ."


In [38]:
data.describe() #check lại data sau khi đã xử lý

,error,correct
count,1519886,1519886
unique,1519242,1514290
top,,access all records of - - now !
freq,165,75


In [39]:
data = data.drop_duplicates() #xóa các dòng trùng lặp
data = data.dropna() #xóa các dòng có giá trị null

In [40]:
data.describe() #check lại data sau khi đã xử lý

,error,correct
count,1519621,1519621
unique,1519242,1514290
top,,"this page was last edited on april , at ."
freq,163,71


In [41]:
data['error'] = data['error'].fillna("")
data['correct'] = data['correct'].fillna("")


In [42]:
for i, row in enumerate(data['error']): #check từng câu trong cột error
  if row.split(" ") == data['correct'].iloc[i].split(" "): #nếu câu trong cột error giống câu trong cột correct
    data.iloc[i] = np.nan #thì gán giá trị null cho dòng đó

In [43]:
data = data.dropna() #loại bỏ các dòng có giá trị null

In [44]:
data.describe() #các dòng giống nhau ở cả 2 cột đã bị loại bỏ

,error,correct
count,1489685,1489685
unique,1489333,1484698
top,,"this page was last edited on april , at ."
freq,162,71


In [45]:
import pandas as pd #thư viện dùng để tạo ma trận dữ liệu
import nltk #thư viện dùng để tokenization
nltk.download("punkt")#tải dữ liệu cho tokenization (bộ phân tách từ)
nltk.download("punkt_tab")
def identify_errors(row): #so sánh câu sai và đúng => tìm ra lỗi
    error_tokens = nltk.word_tokenize(row['error']) #chia câu lỗi thành từng token (mỗi token là 1 từ)
    correct_tokens = nltk.word_tokenize(row['correct']) #chia câu đúng thành từng token (mỗi token là 1 từ)
    return [word for word in error_tokens if word not in correct_tokens] #trả về các từ không nằm trong câu đúng nhưng có trong câu sai

data['identified_errors'] = data.apply(identify_errors, axis=1) #áp dụng hàm identify_errors cho từng dòng trong dataframe, tạo 1 cột mới là identified_errors chứa các từ sai

error_frequency = data['identified_errors'].explode().value_counts() #đếm số lần xuất hiện của từng từ sai

print(error_frequency.head(10)) #in ra 10 từ sai xuất hiện nhiều nhất

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


identified_errors
is     89396
in     83868
of     82204
to     73382
-      64037
a      51946
'      47445
for    41767
on     38235
it     37524
Name: count, dtype: int64


In [96]:
data.head(10)

,error,correct,identified_errors
0,"bitcoin is for , this morning , which coindesk says .","bitcoin goes for , this morning , according to coindesk .","[is, which, says]"
1,the effect of widespread dud targets two face up attack position monsters on the field .,. the effect of widespread dud targets two face up attack position monsters on the field .,[]
3,much many brands and sellers still in the market .,many brands and sellers still in the market .,[much]
4,this is is the latest maintenance release of samba .,this is is the latest maintenance release of samba . .,[]
5,"fairy or not , i am the godmother no just look , but my outfit for taking the part as godmother .","fairy or not , i am the godmother not just a look , but my outfit for taking on the role as godmother .","[no, part]"
6,watcch as this dodge challenger hellcat gets smoked by a tesla model s - with the drag strip .,watch as this dodge challenger hellcat gets smoked by a tesla model s at the drag strip .,"[watcch, -, with]"
7,"momover , these devices have been proven to help consumers during another company his information .","moreover , these devices are proven to help consumers while another company that information .","[momover, have, been, during, his]"
9,worthless involved is supporting for the movement .,get involved and help the movement !,"[worthless, is, supporting, for, .]"
10,mark mohler said in a post on instagram that he and fellow diver kimberley jeffries have confirmed the identity of wonka bee specilis deep blue on wednesday .,"on wednesday , diver mark mohler said in a post on instagram he and fellow diver kimberley jeffries confirmed the identity of the shark as deep blue .","[that, have, wonka, bee, specilis]"
11,"once done , you could pay for your pumpkins and then leave or take advantage of the great stuff in there .","once done , you could pay for your pumpkins and then leave or take advantage of the great stuff there .",[in]


In [46]:
data['correct'] = data['correct'].fillna("")
data['error'] = data['error'].fillna("")
len_1 = [len(val) for val in data['correct']]
len_2 = [len(val) for val in data['error']]


# GEC_Baseline_Attention

In [47]:
import re
import datetime
import numpy as np
import pandas as pd
import random
from tqdm import tqdm
import keras
import tensorflow as tf
from sklearn.metrics import fbeta_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import sentence_bleu
import seaborn as sns
import matplotlib.pyplot as plt

In [56]:
pd.options.display.max_colwidth = 500 #đặt giới hạn chiều rộng của cột để display dataframe
data[:50]

,error,correct,identified_errors
0,"bitcoin is for , this morning , which coindesk says .","bitcoin goes for , this morning , according to coindesk .","[is, which, says]"
1,the effect of widespread dud targets two face up attack position monsters on the field .,. the effect of widespread dud targets two face up attack position monsters on the field .,[]
3,much many brands and sellers still in the market .,many brands and sellers still in the market .,[much]
4,this is is the latest maintenance release of samba .,this is is the latest maintenance release of samba . .,[]
5,"fairy or not , i am the godmother no just look , but my outfit for taking the part as godmother .","fairy or not , i am the godmother not just a look , but my outfit for taking on the role as godmother .","[no, part]"
6,watcch as this dodge challenger hellcat gets smoked by a tesla model s - with the drag strip .,watch as this dodge challenger hellcat gets smoked by a tesla model s at the drag strip .,"[watcch, -, with]"
7,"momover , these devices have been proven to help consumers during another company his information .","moreover , these devices are proven to help consumers while another company that information .","[momover, have, been, during, his]"
9,worthless involved is supporting for the movement .,get involved and help the movement !,"[worthless, is, supporting, for, .]"
10,mark mohler said in a post on instagram that he and fellow diver kimberley jeffries have confirmed the identity of wonka bee specilis deep blue on wednesday .,"on wednesday , diver mark mohler said in a post on instagram he and fellow diver kimberley jeffries confirmed the identity of the shark as deep blue .","[that, have, wonka, bee, specilis]"
11,"once done , you could pay for your pumpkins and then leave or take advantage of the great stuff in there .","once done , you could pay for your pumpkins and then leave or take advantage of the great stuff there .",[in]


In [57]:
data.shape

(1489685, 3)

In [58]:
#loại bỏ các dòng chứa các câu có độ dài lớn hơn 30 từ trong data (bất kể trong cột nào)
data = data[~data['error'].str.split().apply(lambda x: len(x) > 30)] #loại bỏ các câu có độ dài lớn hơn 30 từ trong cột error
data = data[~data['correct'].str.split().apply(lambda x: len(x) > 30)] #loại bỏ các câu có độ dài lớn hơn 30 từ trong cột correct

In [113]:
data.to_csv("cleaned_data.csv", index=False) #lưu dataframe đã xử lý vào file csv

In [115]:
data = pd.read_csv("cleaned_data.csv") #đọc lại file csv đã lưu

In [116]:
data.shape

(1398232, 3)

In [59]:
def preprocess(t, add_start_token, add_end_token):
  t = str(t)
  if add_start_token == True and add_end_token == False: #nếu thêm start token và không thêm end token
    t = '<start>'+' '+t
  if add_start_token == False and add_end_token == True: #nếu không thêm start token và thêm end token
    t = t+' '+'<end>'
  if add_start_token == True and add_end_token == True: #nếu thêm cả start token và end token
    t = '<start>'+' '+t+' '+'<end>'

  t = re.sub(' +', ' ', t) #loại bỏ khoảng trắng thừa
  return t

In [60]:
encoder_input = [preprocess(line, add_start_token= True, add_end_token=True) for line in data['error']]
decoder_input = [preprocess(line, add_start_token= True, add_end_token=False) for line in data['correct']]
decoder_output = [preprocess(line, add_start_token= False, add_end_token=True) for line in data['correct']]

In [61]:
print(encoder_input[0]) # in ra dòng đầu tiên của encoder_input
print(decoder_input[0]) 
print(decoder_output[0]) #output của ecoder output
print(len(encoder_input), len(decoder_input), len(decoder_output)) #số lượng dòng trong 3 cột

<start> bitcoin is for , this morning , which coindesk says . <end>
<start> bitcoin goes for , this morning , according to coindesk .
bitcoin goes for , this morning , according to coindesk . <end>
1398232 1398232 1398232


In [62]:
print(data.shape)

(1398232, 3)


In [54]:
#loại bỏ các câu có độ dài lớn hơn 30 từ trong câu
max_len = 30
encoder_input = [line for line in encoder_input if len(line.split()) <= max_len]
decoder_input = [line for line in decoder_input if len(line.split()) <= max_len]
decoder_output = [line for line in decoder_output if len(line.split()) <= max_len]
print(len(encoder_input), len(decoder_input), len(decoder_output)) #số lượng dòng trong 3 cột sau khi loại bỏ các câu có độ dài lớn hơn 30 từ

1365932 1381814 1381814


In [63]:
tokenizer = Tokenizer(filters='', split=" ")
tokenizer.fit_on_texts(encoder_input)
word_index = tokenizer.word_index #tạo index cho token

max_length = max([ len(row.split(" ")) for row in encoder_input ]) #tìm độ dài lớn nhất của câu trong encoder_input
INPUT_ENCODER_LENGTH = max_length 

enc_input_encoded = tokenizer.texts_to_sequences(encoder_input) #tokenize các từ thành dạng index
enc_input_padded= pad_sequences(enc_input_encoded, maxlen=INPUT_ENCODER_LENGTH, padding="post") #đệm các câu để có cùng độ dài thành câu lớn nhất

print(enc_input_padded.shape)

(1398232, 32)


In [64]:
print(INPUT_ENCODER_LENGTH)

32


In [79]:
print(encoder_input[0]) #in ra câu đầu tiên của encoder_input
print(enc_input_padded[0]) #sau khi được chuyển sang index và padding

<start> And he took in my favorite subject like soccer . <end>
[  1  10  45 183  11  12 342 966  41 768   3   2   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [65]:
decoder_data = decoder_input.copy() #copy decoder_input sang decoder_data
decoder_data.extend(decoder_output) #nối decoder_output vào cuối decoder_data

out_tokenizer = Tokenizer(filters='', split=" ") #tạo tokenizer cho decoder_data
out_tokenizer.fit_on_texts(decoder_data) #fit dữ liệu vào tokenizer
word_index = out_tokenizer.word_index #tạo index cho từ

max_length = max([ len(row.split(" ")) for row in decoder_input ]) #tìm độ dài lớn nhất của câu trong decoder_input
INPUT_DECODER_LENGTH = max_length

In [67]:
dec_input_encoded = out_tokenizer.texts_to_sequences(decoder_input) #chuyển các từ thành index
dec_input_padded= pad_sequences(dec_input_encoded, maxlen=INPUT_DECODER_LENGTH, padding="post", truncating = "post") #đệm các câu để có cùng độ dài thành câu lớn nhất

print(dec_input_padded.shape)

(1398232, 32)


In [82]:
print(decoder_input[0]) #in ra câu đầu tiên của decoder_input
print(dec_input_padded[0]) #sau khi được chuyển sang index và padding

<start> And he took in my favorite subjects like soccer .
[   3   10   45  191   12   11  328 1399   43  754    1    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]


In [68]:
dec_output_encoded = out_tokenizer.texts_to_sequences(decoder_output) #chuyển các từ thành index
dec_output_padded= pad_sequences(dec_output_encoded, maxlen=INPUT_DECODER_LENGTH, padding="post", truncating = "post") #đệm các câu để có cùng độ dài thành câu lớn nhất

print(dec_output_padded.shape)

(1398232, 32)


In [84]:
print(decoder_output[1])
print(dec_output_padded[1])

Actually , he was the one who let me know about Lang - . . <end>
[278   5  45  17   6  56  83 352  26  72  36 192  40   1   1   4   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0]


In [69]:
import io #thư viện dùng để xử lý file

def load_vectors(fname): #hàm load vector từ file
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore') #mở file
    n, d = map(int, fin.readline().split()) #đọc số lượng từ và số chiều của từ
    data = {} #tạo dictionary lưu từ và vector
    for line in fin: #đọc từng dòng trong file
        tokens = line.rstrip().split(' ') #tách từng từ và vector
        data[tokens[0]] = np.asarray(tokens[1:]) #lưu từ và vector vào dictionary
    return data

In [ ]:
!pip install gensim 

   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/24.0 MB ? eta -:--:--
   -----------------

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
embedding_index = load_vectors(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Embedding Pretrained\fastext.vec") #load vector từ file

In [ ]:
word_index = out_tokenizer.word_index
num_tokens = len(word_index) + 2
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)

    if type(embedding_vector) == np.ndarray and embedding_vector.shape[0] == 300:
        embedding_matrix[i] = embedding_vector
        hits += 1

    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))
np.save(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\in_embedding_len30.npy", embedding_matrix)

Converted 152066 words (250691 misses)


In [ ]:
word_index = out_tokenizer.word_index
num_tokens = len(word_index) + 2
embedding_dim = 300
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)

    if type(embedding_vector) == np.ndarray and embedding_vector.shape[0] == 300:
        embedding_matrix[i] = embedding_vector
        hits += 1

    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))
np.save(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\out_embedding_len30.npy", embedding_matrix)

Converted 152066 words (250691 misses)


In [ ]:
in_embedding_matrix = np.load(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\in_embedding.npy")
out_embedding_matrix = np.load(r"C:\Users\Admin\Downloads\Model_NLP\NLP\Model Attachment\out_embedding.npy")
print(in_embedding_matrix.shape, out_embedding_matrix.shape)

(402759, 300) (402759, 300)


In [98]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class Encoder(nn.Module):
    def __init__(self, inp_vocab_size, embedding_size, lstm_size, input_length):
        super(Encoder, self).__init__()
        self.vocab_size = inp_vocab_size #số lượng từ vựng
        self.embedding_size = embedding_size
        self.lstm_units = lstm_size
        self.input_length = input_length

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(in_embedding_matrix), freeze=True,padding_idx=0)
        self.lstm = nn.LSTM(self.embedding_size, self.lstm_units,batch_first=True)
    def forward(self,input_sequence,states):
        input_embedding = self.embedding(input_sequence)
        self.lstm_output, (self.state_h, self.state_c) = self.lstm(input_embedding, states)
        return self.lstm_output, self.state_h, self.state_c
    def initialize_states(self,batch_size):
        lstm_state_h = torch.zeros((1,batch_size,self.lstm_units)).to(device)
        lstm_state_c = torch.zeros((1,batch_size,self.lstm_units)).to(device)
        return lstm_state_h, lstm_state_c

In [99]:
def dot_func(encoder_output,decoder_hidden_state):
    decoder_hidden_state = torch.reshape(decoder_hidden_state,[decoder_hidden_state.shape[1],1,decoder_hidden_state.shape[2]])
    dot_product = torch.matmul(encoder_output,decoder_hidden_state.transpose(1,2))
    return dot_product

In [100]:
class Attention(nn.Module):
    def __init__(self,scoring_function, att_units):
        super(Attention,self).__init__()
        self.scoring_function = scoring_function
        self.att_units = att_units
        self.timesteps = 0
    def forward(self,decoder_hidden_state,encoder_output):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        encoder_output = encoder_output.to(device)
        decoder_hidden_state = decoder_hidden_state.to(device)
        if self.scoring_function == 'dot':
            alpha = F.softmax(dot_func(encoder_output,decoder_hidden_state),dim = 1)
            c_t = torch.sum(alpha *encoder_output,dim = 1)
            return c_t,alpha


In [77]:
class One_Step_Decoder(nn.Module):
    def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
        super(One_Step_Decoder,self).__init__()

        self.tar_vocab_size = tar_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units
        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(out_embedding_matrix), freeze=True,padding_idx=0)
        self.lstm = nn.LSTM(self.dec_units + self.embedding_dim,self.dec_units,batch_first=True)
        self.dense = nn.Linear(self.dec_units,self.tar_vocab_size)
    def forward(self,input_to_decoder,encoder_output,state_h,state_c):
        input_embedding = self.embedding(input_to_decoder)
        if self.score_fun == "dot":
            attention = Attention("dot",self.att_units)
            context_vector,attention_weights = attention(state_h,encoder_output)
        out = torch.cat([input_embedding,context_vector.unsqueeze(1)],dim = 2)
        self.lstm_output, (self.state_h, self.state_c_) = self.lstm(out, (state_h,state_c))
        result_out = self.dense(self.lstm_output)
        return result_out.squeeze(1), self.state_h, self.state_c_, attention_weights, context_vector

In [101]:
class Decoder(nn.Module):
    def __init__(self,out_vocab_size,embedding_dim,input_length,dec_units, score_fun,att_units):
        super(Decoder,self).__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_dim = embedding_dim
        self.input_length = input_length
        self.dec_units = dec_units
        self.score_fun = score_fun
        self.att_units = att_units
        self.one_step_decoder = One_Step_Decoder(self.out_vocab_size, self.embedding_dim, self.input_length, self.dec_units ,self.score_fun ,self.att_units)
    def forward(self,input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_to_decoder = input_to_decoder.to(device)
        encoder_output = encoder_output.to(device)
        decoder_hidden_state = decoder_hidden_state.to(device)
        decoder_cell_state = decoder_cell_state.to(device)

        batch_size, timesteps = input_to_decoder.shape
        out_array = torch.zeros((batch_size, timesteps, self.one_step_decoder.tar_vocab_size)).to(device)

        for timestep in range(timesteps):
            output,decoder_hidden_state,decoder_cell_state,_,_ = self.one_step_decoder(input_to_decoder[:,timestep:timestep+1],encoder_output,decoder_hidden_state,decoder_cell_state)
            out_array[:,timestep,:] = output
        return out_array

In [102]:
class encoder_decoder(nn.Module):
    def __init__(self,enc_units,dec_units,scoring_func,att_units):
        super(encoder_decoder,self).__init__()
        self.scoring_func = scoring_func
        self.att_units = att_units
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.encoder = Encoder(INPUT_VOCAB_SIZE, embedding_size = 300, lstm_size= self.enc_units , input_length= INPUT_ENCODER_LENGTH)
        self.decoder = Decoder(OUTPUT_VOCAB_SIZE, embedding_dim=300, input_length = None, dec_units= self.dec_units ,score_fun =self.scoring_func,att_units = self.att_units)
    
    def forward(self,data):
        input,output = data[0],data[1]
        states = self.encoder.initialize_states(input.shape[0])
        encoder_output,encoder_final_state_h,encoder_final_state_c = self.encoder(input, states)
        decoder_output = self.decoder(output,encoder_output, encoder_final_state_h, encoder_final_state_c)
        return decoder_output

In [103]:
INPUT_VOCAB_SIZE = len(list(tokenizer.word_index)) +2 #kích thước từ vựng đầu vào +2 là do thêm start token và end token
OUTPUT_VOCAB_SIZE = len(list(out_tokenizer.word_index)) +2 #tương tự như trên
print(INPUT_VOCAB_SIZE, INPUT_ENCODER_LENGTH, OUTPUT_VOCAB_SIZE, INPUT_DECODER_LENGTH) #in ra kích thước tập từ vựng, đọ dài câu, kích thước tập từ vựng đầu ra, độ dài câu đầu ra, kích thước batch

640324 32 402759 32


In [104]:
import torch
import torch.nn as nn
class Seq2SeqDataset(torch.utils.data.Dataset):
    def __init__(self,enc_input_padded,dec_input_padded,dec_output_padded):
        self.enc_input_padded = torch.tensor(enc_input_padded)
        self.dec_input_padded = torch.tensor(dec_input_padded)
        self.dec_output_padded = torch.tensor(dec_output_padded)
    def __len__(self):
        return len(self.enc_input_padded)
    def __getitem__(self,idx):
        return self.enc_input_padded[idx],self.dec_input_padded[idx],self.dec_output_padded[idx]


dataset = Seq2SeqDataset(enc_input_padded,dec_input_padded,dec_output_padded)
test_size = int(len(dataset)*0.3)
train_size = len(dataset) - test_size
train_dataset,test_dataset = torch.utils.data.random_split(dataset,[train_size,test_size])

In [105]:
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size = 32,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size = 32,shuffle = False)

In [111]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [110]:
for a,b,c in train_loader:
    print(a.shape)
    print(b.shape)
    print(c.shape)
    break
#thử cho encoder_input_padded đi qua lớp embedding
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a,b,c = next(iter(train_loader))
b = torch.randn(32, 1, 256, device=device)  # PHẢI có dòng này
b = torch.randint(0, 1000, (32,1 ,256)).to(device).float()
c= torch.matmul(a,b.transpose(1,2))
print("Alo", c.shape)
alpha = F.softmax(c, dim=1)
print(torch.sum(alpha * a, dim=1).shape)
#thử lớp LSTM
embedding = nn.Embedding.from_pretrained(torch.FloatTensor(in_embedding_matrix), freeze=True,padding_idx=0).to(device)
a = torch.randint(0, 1000, (32,32)).to(device).long()
print(embedding(a).shape)
lstm = nn.LSTM(300,256,batch_first=True).to(device)
lstm_output, (state_h, state_c) = lstm(embedding(a))
print(lstm_output.shape, state_h.shape, state_c.shape)

context_vecto = torch.randn(size = (32, 256)).to(device)
context_vecto = context_vecto.unsqueeze(1)
print(context_vecto.shape)
input_embedding = torch.randn(size = (32, 1, 300)).to(device)
print(torch.cat([input_embedding,context_vecto],dim =2).shape)

torch.Size([32, 32])
torch.Size([32, 32])
torch.Size([32, 32])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [112]:
model = encoder_decoder(enc_units=512,dec_units=512,scoring_func="dot",att_units=256)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [85]:
#tổng tham số của mô hình
total_params = sum(p.numel() for p in model.parameters())

#tổng tham số huấn luyện
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Tham số của mô hình: {total_params}")
print(f"Tham số huấn luyện: {trainable_params}")

Tham số của mô hình: 452653487
Tham số huấn luyện: 210998087


In [109]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, path='len30_checkpoint.pt'):
        self.patience = patience
        self.delta = delta
        self.path = path
        self.best_loss = float('inf')
        self.counter = 0

    def __call__(self, val_loss, model,epoch):
        if val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model.state_dict(), self.path)  # Lưu model tốt nhất
            print(f"✅ Model từ epoch {epoch+1} đã được lưu tại {self.path}.")
        else:
            self.counter += 1
            print(f"❌ Model từ epoch {epoch+1} không được lưu.")
            if self.counter >= self.patience:
                print("Early stopping triggered!")
                return True
        return False

In [116]:
#đăng nhập vào wandb
import wandb
wandb.login(key = "1a1e9d904fb11812f635b8c3f9a93ae09da4cd04")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Admin\_netrc


True

In [ ]:
wandb.init(project="project_nlp_len30_new", name="second_run", config={"epochs": 10, "lr": 1e-3,"time":datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")})

In [ ]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm  # Import tqdm để hiển thị tiến trình

def train_model(model, train_loader, val_loader, num_epochs=10, lr=1e-3, patience=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)
    early_stopping = EarlyStopping(patience=patience)
    loss_function = nn.CrossEntropyLoss(ignore_index=0)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0

        # Thanh tiến trình cho Training
        train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Training]", leave=False)

        for enc_input_padded, dec_input_padded, dec_output_padded in train_progress:
            optimizer.zero_grad()

            enc_input_padded = enc_input_padded.to(device)
            dec_input_padded = dec_input_padded.to(device)
            dec_output_padded = dec_output_padded.to(device)

            outputs = model([enc_input_padded, dec_input_padded])
            outputs = outputs.view(-1, outputs.size(-1))  # (batch_size * seq_len, vocab_size)
            targets = dec_output_padded.view(-1)  # (batch_size * seq_len)

            loss = loss_function(outputs.float(), targets.long())
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_progress.set_postfix(loss=loss.item())
        train_loss /= len(train_loader)

        # ----- Validation -----
        model.eval()
        val_loss = 0

        val_progress = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Validation]", leave=False)

        with torch.no_grad():
            for enc_input_padded, dec_input_padded, dec_output_padded in val_progress:
                enc_input_padded = enc_input_padded.to(device)
                dec_input_padded = dec_input_padded.to(device)
                dec_output_padded = dec_output_padded.to(device)

                outputs = model([enc_input_padded, dec_input_padded])
                outputs = outputs.view(-1, outputs.size(-1))
                targets = dec_output_padded.view(-1)

                loss = loss_function(outputs.float(), targets.long())
                val_loss += loss.item()
                val_progress.set_postfix(val_loss=loss.item())

        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")

        # Log thông số lên WandB
        wandb.log({"epoch": epoch+1, "train_loss": train_loss, "val_loss": val_loss, "learning_rate": optimizer.param_groups[0]["lr"]})

        scheduler.step(val_loss)

        # Lưu mô hình nếu có cải thiện
        if early_stopping(val_loss, model, epoch):
            wandb.run.summary["best_val_loss"] = val_loss
            torch.save(model.state_dict(), "len30_best_model.pth")
            wandb.save("best_model.pth")
            break

    # Kết thúc phiên làm việc với WandB
    wandb.finish()

In [ ]:
#train model
train_model(model, train_loader, test_loader, num_epochs=10, lr=1e-3, patience=5)

Epoch 1/20 - Train Loss: 2.4446 - Val Loss: 1.7339
✅ Model từ epoch 1 đã được lưu tại len30_checkpoint.pt.


Epoch 2/20 - Train Loss: 1.5060 - Val Loss: 1.5012
✅ Model từ epoch 2 đã được lưu tại len30_checkpoint.pt.


Epoch 3/20 - Train Loss: 1.2795 - Val Loss: 1.4306
✅ Model từ epoch 3 đã được lưu tại len30_checkpoint.pt.


Epoch 4/20 - Train Loss: 1.1531 - Val Loss: 1.3983
✅ Model từ epoch 4 đã được lưu tại len30_checkpoint.pt.


Epoch 5/20 - Train Loss: 1.0696 - Val Loss: 1.3853
✅ Model từ epoch 5 đã được lưu tại len30_checkpoint.pt.


...

Epoch 8/20 - Train Loss: 0.9372 - Val Loss: 1.3872
❌ Model từ epoch 8 không được lưu.


Epoch 9/20 - Train Loss: 0.9127 - Val Loss: 1.3832
❌ Model từ epoch 9 không được lưu.


Epoch 10/20 - Train Loss: 0.8874 - Val Loss: 1.3810
❌ Model từ epoch 10 không được lưu.


Epoch 11/20 - Train Loss: 0.8694 - Val Loss: 1.3837
❌ Model từ epoch 11 không được lưu.


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(),"epoch_10_len30_final_model.pth") #lưu model cuối cùng

In [105]:
model.load_state_dict(torch.load(r"C:\Users\Admin\Downloads\NLP\best_model.pth"))
model.to(device)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22052\2455360464.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r"C:\Users\Admin\Downloads\

encoder_decoder(
  (encoder): Encoder(
    (embedding): Embedding(52381, 300, padding_idx=0)
    (lstm): LSTM(300, 512, batch_first=True)
  )
  (decoder): Decoder(
    (one_step_decoder): One_Step_Decoder(
      (embedding): Embedding(41061, 300, padding_idx=0)
      (lstm): LSTM(812, 512, batch_first=True)
      (dense): Linear(in_features=512, out_features=41061, bias=True)
    )
  )
)

In [101]:
model.eval()

encoder_decoder(
  (encoder): Encoder(
    (embedding): Embedding(52381, 300, padding_idx=0)
    (lstm): LSTM(300, 512, batch_first=True)
  )
  (decoder): Decoder(
    (one_step_decoder): One_Step_Decoder(
      (embedding): Embedding(41061, 300, padding_idx=0)
      (lstm): LSTM(812, 512, batch_first=True)
      (dense): Linear(in_features=512, out_features=41061, bias=True)
    )
  )
)